# **SFv2 Template notebook**

## Import modules

In [1]:
import os
import sys

data_path=os.path.abspath('C:/Users/hcarv/Documents/work/Protein Design 2021/Group1') #Where your group data is
base_path=os.path.abspath('C:/Users/hcarv/Documents/GitHub/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import Protocols as P

from simtk.unit import *


## Set defaults

In [2]:
std_report=100

CALB_variants={('A',187): 'ASP', 
                       ('A',224): 'HID'}

SETD2_variants={('A',1499): 'CYX', 
                    ('A',1501): 'CYX', 
                    ('A',1516): 'CYX', 
                    ('A',1520): 'CYX', 
                    ('A',1529): 'CYX', 
                    ('A',1533):'CYX', 
                    ('A',1539):'CYX', 
                    ('A',1631):'CYX', 
                    ('A',1678):'CYX', 
                    ('A',1680):'CYX', 
                    ('A',1685):'CYX', 
                    ('B',36): 'LYN'} 

CALB_defaults={'ensemble': 'NPT',
               'step': 1*picoseconds, 
               'report': std_report, 
               'restrained_sets': {'selections': ['protein and backbone'],
                                   'forces': [100*kilojoules_per_mole/angstroms]}}

SETD2_defaults={'ensemble': 'NPT',
                'step': 100*picoseconds, 
                'report': std_report, 
                'restrained_sets': {'selections': ['protein and name CA and not chainid 1', 'chainid 3 and resid 0'],
                                    'forces': [100*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms]}}

# **Protocols**

## 1. From PDB to openMM: pdb2omm

In [3]:
pdb='5a71a.pdb'

importlib.reload(P)
sim_tools=P.Protocols(workdir=data_path)

system_omm=sim_tools.pdb2omm(input_pdbs=pdb,
                            extra_ff_files=['gaff-2.11modv3_1.xml'], 
                            protonate=True,
                            solvate=True,
                            inspect=True,
                            pH_protein=7.0,
                            residue_variants=CALB_variants) 





Extra force field file 1: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/gaff-2.11modv3_1.xml
Using default force fields: ('amber14-all.xml', 'amber14/tip4pew.xml')
PDB file generated: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/system.pdb


## 2. Set simulations

In [4]:
system_omm.setSimulations(dt = 0.002*picoseconds, 
                          temperature = 300*kelvin, 
                          friction = 1/picosecond,
                          equilibrations=[CALB_defaults],
                          pressure=1*atmospheres)

<simtk.openmm.openmm.Platform; proxy of <Swig Object of type 'OpenMM::Platform *' at 0x00000241BF885960> >
Setting equilibration protocol:
	ensemble: NPT
	step: 1 ps
	report: 100
	restrained_sets: {'selections': ['protein and backbone'], 'forces': [Quantity(value=100, unit=kilojoule/(angstrom*mole))]}
Steps is not unitless: 1 ps
Converted to unitless using integration time of 0.002 ps: 500


## **Run simulations**

1. Energy minimization
2. Equilibration(s)
3. Production(s)

In [5]:
system_omm.run_energyMinimization()
system_omm.run_equilibrations()
#TODO: system_omm.run_productions()

Performing energy minimization: -996161.5042478982 kJ/mol
System is now minimized: -1270673.5042478982 kJ/mol
PDB file generated: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/Emin.pdb
Run: NPT
Restrained NPT equilibration (1.0 ps)...
NPT equilibration finished.
PDB file generated: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/EQ_NPT.pdb
System is now equilibrated (?): -1108031.5042478982 kJ/mol


<module 'simtk.openmm.app.simulation' from 'C:\\Users\\hcarv\\anaconda3\\lib\\site-packages\\simtk\\openmm\\app\\simulation.py'>